In [1]:
!pip install -q geopandas;
!pip install -q geemap;
!pip install -q --upgrade folium;

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.6/98.6 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.5/130.5 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 23.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 15.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.5/46.5 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 30.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 31.1 MB/s eta 0:00:00


In [2]:
import json
import ee, datetime, folium
import pandas as pd
from IPython.display import Image
from pylab import *
import matplotlib.pyplot as plt
import seaborn as sns
import geopandas as gpd
from folium.plugins import HeatMap
import io
import fiona
import branca.colormap as cm
import geemap
import os
from shapely.geometry import shape
from shapely.geometry import shape, Polygon, MultiPolygon

# Extract Satellite Indices per Geographic Area in England

## First authenticate GEE

In [3]:
# Trigger the authentication flow.
ee.Authenticate()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=Xcry9CJAe2LH19vVkjrUziVpnrgFnLtyN_JskfIandw&tc=_76nRAPK8zS2E48u5piBqrB48LLakdEE0BFKm0hgX6s&cc=raqcEWo_KzmO2q0eBvbxKlY5wP83CvetGfr4VcKIvZU

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1Adeu5BVMzlBHGWjUiYfCIAcHC9X4THYbc-bxe4QGwJuf2e9iWncion4ZvlQ

Successfully saved authorization token.


In [4]:
# Initialize the library.
PROJECT_NAME = 'ee-env-data-extractor' ## TODO: CHANGE TO YOUR PROJECT NAME

ee.Initialize(project=PROJECT_NAME)

## Then also connect to Google Drive


In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Global Variables

In [6]:
your_location = 'ENV_DATA_EXTRACTOR' ## TODO: CHANGE TO YOUR GDRIVE LOCATION
project_folder = f"drive/MyDrive/{your_location}/environmental_data_extractor/Colab_Notebooks/"

In [7]:
SELECTORS = {
    'CAMS' :  ['total_aerosol_optical_depth_at_550nm_surface', 'particulate_matter_d_less_than_25_um_surface'],
    'S5' : ['NO2_column_number_density'],
    'OMI' : ['ozone'],
}


SCALE = {
    'CAMS' :  5000,
    'S5' : 1000,
    'OMI' : 1000,
}

## 0 Functions

In [8]:
# Define a method for displaying Earth Engine image tiles to a folium map.
def add_ee_layer(self, ee_image_object, vis_params, name, show=True, opacity=1, min_zoom=0):
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
        name=name,
        show=show,
        opacity=opacity,
        min_zoom=min_zoom,
        overlay=True,
        control=True
        ).add_to(self)

# Add the Earth Engine layer method to folium.
folium.Map.add_ee_layer = add_ee_layer

In [9]:
# Define a function for displaying shapefiles saved into a feature collection
def plot_ee_feature_shapefile_folium(ee_feature, zoom_start=7):
  # Get the bounds of the feature collection
  aoi = ee_feature.geometry()

  # Create a folium map object.
  center = aoi.centroid(10).coordinates().reverse().getInfo()
  m = folium.Map(location=center, zoom_start=zoom_start)

  # Add layers to the folium map.
  # Add the feature collection to the map as a GeoJSON layer
  m.add_child(folium.GeoJson(data=ee_feature.getInfo()))

  # Add a layer control panel to the map.
  m.add_child(folium.LayerControl())

  # Display the map.
  display(m)

In [10]:
def plot_ee_feature_shapefile_geemap(fc, zoom_start=6, coordinates=(-0.1278, 51.5074), color=None):

  # Create a Map object
  Map = geemap.Map() # set initial center and zoom level

  # Add the Feature Collection to the Map, centered on the shapefile
  Map.addLayer(fc, {}, 'Shapefile')

  Map.centerObject(fc, zoom_start)

  if color:
    # Add a color based on the 'means' property
    colorized = fc.map(lambda feature: feature.set({'style': {'color': 'blue', 'fillColor': feature.get(color)}}))

  # Add the colorized feature collection to the map
  Map.addLayer(colorized, {}, 'Colorized')

  display(Map)

In [11]:
def spatial_join_ee_fc(ee_fc_fine, ee_fc_coarse):

  # Define the spatial join relationship
  spatialFilter = ee.Filter.intersects(
    leftField='.geo', # geometry field in regions feature collection
    rightField='.geo', # geometry field in wards/lsoas feature collection
    maxError=10) # tolerance distance in meters

  # Perform the spatial join using the defined spatial filter
  ee_fc_joined = ee.Join.simple().apply(
    primary=ee_fc_fine,
    secondary=ee_fc_coarse,
    condition=spatialFilter)

  return ee_fc_joined

In [12]:
def get_satellite_imagery_for_AQ(ee_the_region, sat, START_DATE, END_DATE):

  # # params
  # START_DATE = '2021-01-01' # inclusive
  # END_DATE = '2022-01-01' # exclusive
  # CLOUD_FILTER = 1 # maximum amount of clouds in percent

  params = SELECTORS[sat]

  # these are parameters for selecting the Sentinel Image
  aoi = ee_the_region.geometry()

  if sat == 'S5':
    # # params
    # START_DATE = '2021-01-01' # inclusive
    # END_DATE = '2022-01-01' # exclusive
    # CLOUD_FILTER = 1 # maximum amount of clouds in percent

    # Load Sentinel-5P NO2 data
    im_dat = ee.ImageCollection('COPERNICUS/S5P/NRTI/L3_NO2')

    # Filter data by date and location
    filtered = im_dat.filterDate(START_DATE, END_DATE).filterBounds(aoi)

    # Calculate mean NO2 concentration
    mean_concentration = filtered.select(params).mean()

  elif sat == 'OMI':
    # # params
    # START_DATE = '2011-01-01' # inclusive
    # END_DATE = '2012-01-01' # exclusive
    # CLOUD_FILTER = 1 # maximum amount of clouds in percent

    # Load the dataset
    im_dat = ee.ImageCollection('TOMS/MERGED')

    # Filter data by date and location
    filtered = im_dat.filterDate(START_DATE, END_DATE).filterBounds(aoi)

    # Calculate mean
    mean_concentration = filtered.select(params).mean()

  elif sat == 'CAMS':
    # # params
    # START_DATE = '2016-06-01' # inclusive
    # END_DATE = '2017-06-01' # exclusive
    # CLOUD_FILTER = 1 # maximum amount of clouds in percent

    # Load data
    im_dat = ee.ImageCollection('ECMWF/CAMS/NRT')

    # Filter data by date and location
    filtered = im_dat.filterDate(START_DATE, END_DATE).filterBounds(aoi)

    # Calculate mean concentration
    mean_concentration = filtered.select(params).mean()

  return mean_concentration

In [13]:
def efficent_fc_to_gdf(fc, selectors, batch_size=2000):

  print ("running efficient conversion to gdf")

  # Get the total number of features in the feature collection
  num_features = fc.size().getInfo()

  # Define a list to store the GeoDataFrames
  gdfs = []

  # Loop over the feature collection in batches of size batch_size
  for i in range(0, num_features, batch_size):
      print ("running efficient conversion to gdf. data {} of {}...".\
              format(i, int(num_features)))
      # Get the next batch of features using the limit() method
      # features = fc.limit(batch_size, str(i))
      features_list = fc.toList(batch_size, i)
      features = ee.FeatureCollection(features_list)

      # Convert the batch of features to a GeoDataFrame
      gdf = geemap.ee_to_geopandas(features, selectors=selectors)

      # Add the GeoDataFrame to the list
      gdfs.append(gdf)

  # Concatenate the GeoDataFrames into a single GeoDataFrame
  final_gdf = pd.concat(gdfs)

  # Display the GeoDataFrame
  # print(final_gdf.head())

  return final_gdf

In [16]:
def extract_AIQ_lsoa_scores_region(region_name, ee_regions, ee_lsoas_UK, sat, START_DATE, END_DATE):

  year = START_DATE.split("-")[0]
  print("****** Processing {} in {} year *******".format(region_name, year))

  # prepare region geographies
  ee_the_region = ee_regions.filter(ee.Filter.eq('RGN22NM',region_name))
  ee_region_lsoas = spatial_join_ee_fc(ee_lsoas_UK, ee_the_region)
  print ("prepared Region geographies ...")

  # pull Sentinel-5 imagery
  s5_col = get_satellite_imagery_for_AQ(ee_the_region, sat, START_DATE, END_DATE)
  print ("pulled imagery ...")


  # take the Sentinel-5 NO2 we calculated
  S5NO2 = s5_col
  # get the shapefile we need
  shapefile = ee_region_lsoas
  # Get the mean value of each band within the shapefile
  means = S5NO2.reduceRegions(collection=shapefile, reducer=ee.Reducer.mean(), scale=SCALE[sat])
  print ("calculated LSOAs scores ...")

  # a small hack because with 1 param, this gives a column means
  # but with two and more keeps their original names
  if len(SELECTORS[sat]) == 1:
    selectors = ['LSOA21NM', 'LSOA21CD', 'mean']
  else:
    selectors = ['LSOA21NM', 'LSOA21CD'] + SELECTORS[sat]
  # Convert the feature collection to a GeoPandas dataframe
  # gdf_means = geemap.ee_to_geopandas(means, selectors=selectors)
  gdf_means = efficent_fc_to_gdf(means, selectors=selectors)
  # # Define the CRS of the GeoJSON data
  # gdf_means.crs = 'EPSG:4326'
  print ("converted feature collection to geopandas df ...")


  # save csv with results
  csv_results_folder = project_folder + \
         "../results/{}/lsoa/{}/".format(sat, year)

  if not os.path.exists(csv_results_folder):
      os.makedirs(csv_results_folder)

  # save csv with results
  csv_results_address = csv_results_folder + f"{region_name}.csv"

  # create a Pandas dataframe with selected columns
  df_means = pd.DataFrame(gdf_means[selectors])
  df_means.to_csv(csv_results_address, index=False)
  print ("saved csv ...")

  return gdf_means

## 1 Download / Read In Data

## Spatial Data

### Region Data

In [17]:
# England regions data is from here https://geoportal.statistics.gov.uk/
# https://geoportal.statistics.gov.uk/datasets/ons::regions-december-2022-en-bfc/explore?location=52.916331%2C-2.000000%2C7.74

# drive_address = project_folder + "data/England-regions/Regions_(December_2022)_EN_BFC.shp"
asset_address = f"projects/{PROJECT_NAME}/assets/regions"

# 2 we can read the data as a feature shapefile from GEE assets
ee_regions = ee.FeatureCollection(asset_address)

### LSOAs Data

In [18]:
# UK LSOAs data is from here https://geoportal.statistics.gov.uk/
# https://geoportal.statistics.gov.uk/datasets/766da1380a3544c5a7ca9131dfd4acb6_0/explore?location=51.420294%2C-0.114413%2C11.00

# drive_address = project_folder + "data/UK-LSOAs/LSOA_(Dec_2021)_Boundaries_Generalised_Clipped_EW_(BGC).shp"
asset_address = f"projects/{PROJECT_NAME}/assets/lsoas"

# 2 we can read the data as a feature shapefile from GEE assets
ee_lsoas_UK = ee.FeatureCollection(asset_address)

# Process an LSOA

In [21]:
lsoa_N02_scores = extract_AIQ_lsoa_scores_region(region_name='London', ee_regions=ee_regions, ee_lsoas_UK=ee_lsoas_UK,\
                      sat='S5', START_DATE='2022-01-01', END_DATE='2022-02-01')

****** Processing London in 2022 year *******
prepared Region geographies ...
pulled imagery ...
calculated LSOAs scores ...
running efficient conversion to gdf
running efficient conversion to gdf. data 0 of 5128...
running efficient conversion to gdf. data 2000 of 5128...
running efficient conversion to gdf. data 4000 of 5128...
converted feature collection to geopandas df ...
saved csv ...


# Process all LSOAs

2016 -2017

In [ ]:
# for region_name in regions['RGN22NM'].unique():
#     lsoa_NO2_scores = extract_AIQ_lsoa_scores_region(region_name=region_name, ee_regions=ee_regions, ee_lsoas_UK=ee_lsoas_UK,\
#                       sat='S5', START_DATE='2016-01-01', END_DATE='2017-06-22')

In [ ]:
# for region_name in regions['RGN22NM'].unique():
#     lsoa_OMI_scores = extract_AIQ_lsoa_scores_region(region_name=region_name, ee_regions=ee_regions, ee_lsoas_UK=ee_lsoas_UK,\
#                       sat='OMI', START_DATE='2015-01-01', END_DATE='2016-01-01')

In [ ]:
# for region_name in regions['RGN22NM'].unique():
#     lsoa_NO2_scores = extract_AIQ_lsoa_scores_region(region_name=region_name, ee_regions=ee_regions, ee_lsoas_UK=ee_lsoas_UK,\
#                       sat='CAMS', START_DATE='2015-01-01', END_DATE='2016-01-01')

2021 - 2022

In [ ]:
for region_name in regions['RGN22NM'].unique():
    lsoa_CAMS_scores = extract_AIQ_lsoa_scores_region(region_name=region_name, ee_regions=ee_regions, ee_lsoas_UK=ee_lsoas_UK,\
                      sat='CAMS', START_DATE='2021-01-01', END_DATE='2022-01-01')

****** Processing North East in 2021 year *******
prepared Region geographies ...
pulled imagery ...
calculated LSOAs scores ...
running efficient conversion to gdf
running efficient conversion to gdf. data 0 of 1705...
converted feature collection to geopandas df ...
saved csv ...
****** Processing North West in 2021 year *******
prepared Region geographies ...
pulled imagery ...
calculated LSOAs scores ...
running efficient conversion to gdf
running efficient conversion to gdf. data 0 of 4642...
running efficient conversion to gdf. data 2000 of 4642...
running efficient conversion to gdf. data 4000 of 4642...
converted feature collection to geopandas df ...
saved csv ...
****** Processing Yorkshire and The Humber in 2021 year *******
prepared Region geographies ...
pulled imagery ...
calculated LSOAs scores ...
running efficient conversion to gdf
running efficient conversion to gdf. data 0 of 3451...
running efficient conversion to gdf. data 2000 of 3451...
converted feature collecti

In [ ]:
for region_name in regions['RGN22NM'].unique():
    lsoa_OMI_scores = extract_AIQ_lsoa_scores_region(region_name=region_name, ee_regions=ee_regions, ee_lsoas_UK=ee_lsoas_UK,\
                      sat='OMI', START_DATE='2021-01-01', END_DATE='2022-01-01')

****** Processing North East in 2021 year *******
prepared Region geographies ...
pulled imagery ...
calculated LSOAs scores ...
running efficient conversion to gdf
running efficient conversion to gdf. data 0 of 1705...
converted feature collection to geopandas df ...
saved csv ...
****** Processing North West in 2021 year *******
prepared Region geographies ...
pulled imagery ...
calculated LSOAs scores ...
running efficient conversion to gdf
running efficient conversion to gdf. data 0 of 4642...
running efficient conversion to gdf. data 2000 of 4642...
running efficient conversion to gdf. data 4000 of 4642...
converted feature collection to geopandas df ...
saved csv ...
****** Processing Yorkshire and The Humber in 2021 year *******
prepared Region geographies ...
pulled imagery ...
calculated LSOAs scores ...
running efficient conversion to gdf
running efficient conversion to gdf. data 0 of 3451...
running efficient conversion to gdf. data 2000 of 3451...
converted feature collecti

In [ ]:
for region_name in regions['RGN22NM'].unique():
    lsoa_S5_scores = extract_AIQ_lsoa_scores_region(region_name=region_name, ee_regions=ee_regions, ee_lsoas_UK=ee_lsoas_UK,\
                      sat='S5', START_DATE='2021-01-01', END_DATE='2022-01-01')

****** Processing North East in 2021 year *******
prepared Region geographies ...
pulled imagery ...
calculated LSOAs scores ...
running efficient conversion to gdf
running efficient conversion to gdf. data 0 of 1705...
converted feature collection to geopandas df ...
saved csv ...
****** Processing North West in 2021 year *******
prepared Region geographies ...
pulled imagery ...
calculated LSOAs scores ...
running efficient conversion to gdf
running efficient conversion to gdf. data 0 of 4642...
running efficient conversion to gdf. data 2000 of 4642...
running efficient conversion to gdf. data 4000 of 4642...
converted feature collection to geopandas df ...
saved csv ...
****** Processing Yorkshire and The Humber in 2021 year *******
prepared Region geographies ...
pulled imagery ...
calculated LSOAs scores ...
running efficient conversion to gdf
running efficient conversion to gdf. data 0 of 3451...
running efficient conversion to gdf. data 2000 of 3451...
converted feature collecti

In [ ]:
for region_name in regions['RGN22NM'].unique():
    lsoa_CAMS_scores = extract_AIQ_lsoa_scores_region(region_name=region_name, ee_regions=ee_regions, ee_lsoas_UK=ee_lsoas_UK,\
                      sat='CAMS', START_DATE='2019-01-01', END_DATE='2020-01-01')

****** Processing North East in 2019 year *******
prepared Region geographies ...
pulled imagery ...
calculated LSOAs scores ...
running efficient conversion to gdf
running efficient conversion to gdf. data 0 of 1705...
converted feature collection to geopandas df ...
saved csv ...
****** Processing North West in 2019 year *******
prepared Region geographies ...
pulled imagery ...
calculated LSOAs scores ...
running efficient conversion to gdf
running efficient conversion to gdf. data 0 of 4642...
running efficient conversion to gdf. data 2000 of 4642...
running efficient conversion to gdf. data 4000 of 4642...
converted feature collection to geopandas df ...
saved csv ...
****** Processing Yorkshire and The Humber in 2019 year *******
prepared Region geographies ...
pulled imagery ...
calculated LSOAs scores ...
running efficient conversion to gdf
running efficient conversion to gdf. data 0 of 3451...
running efficient conversion to gdf. data 2000 of 3451...
converted feature collecti

In [ ]:
for region_name in regions['RGN22NM'].unique():
    lsoa_OMI_scores = extract_AIQ_lsoa_scores_region(region_name=region_name, ee_regions=ee_regions, ee_lsoas_UK=ee_lsoas_UK,\
                      sat='OMI', START_DATE='2019-01-01', END_DATE='2020-01-01')

****** Processing North East in 2019 year *******
prepared Region geographies ...
pulled imagery ...
calculated LSOAs scores ...
running efficient conversion to gdf
running efficient conversion to gdf. data 0 of 1705...
converted feature collection to geopandas df ...
saved csv ...
****** Processing North West in 2019 year *******
prepared Region geographies ...
pulled imagery ...
calculated LSOAs scores ...
running efficient conversion to gdf
running efficient conversion to gdf. data 0 of 4642...
running efficient conversion to gdf. data 2000 of 4642...
running efficient conversion to gdf. data 4000 of 4642...
converted feature collection to geopandas df ...
saved csv ...
****** Processing Yorkshire and The Humber in 2019 year *******
prepared Region geographies ...
pulled imagery ...
calculated LSOAs scores ...
running efficient conversion to gdf
running efficient conversion to gdf. data 0 of 3451...
running efficient conversion to gdf. data 2000 of 3451...
converted feature collecti

In [ ]:
for region_name in regions['RGN22NM'].unique():
    lsoa_S5_scores = extract_AIQ_lsoa_scores_region(region_name=region_name, ee_regions=ee_regions, ee_lsoas_UK=ee_lsoas_UK,\
                      sat='S5', START_DATE='2019-01-01', END_DATE='2020-01-01')

****** Processing North East in 2019 year *******
prepared Region geographies ...
pulled imagery ...
calculated LSOAs scores ...
running efficient conversion to gdf
running efficient conversion to gdf. data 0 of 1705...
converted feature collection to geopandas df ...
saved csv ...
****** Processing North West in 2019 year *******
prepared Region geographies ...
pulled imagery ...
calculated LSOAs scores ...
running efficient conversion to gdf
running efficient conversion to gdf. data 0 of 4642...
running efficient conversion to gdf. data 2000 of 4642...
running efficient conversion to gdf. data 4000 of 4642...
converted feature collection to geopandas df ...
saved csv ...
****** Processing Yorkshire and The Humber in 2019 year *******
prepared Region geographies ...
pulled imagery ...
calculated LSOAs scores ...
running efficient conversion to gdf
running efficient conversion to gdf. data 0 of 3451...
running efficient conversion to gdf. data 2000 of 3451...
converted feature collecti

In [ ]:
for region_name in regions['RGN22NM'].unique():
    lsoa_CAMS_scores = extract_AIQ_lsoa_scores_region(region_name=region_name, ee_regions=ee_regions, ee_lsoas_UK=ee_lsoas_UK,\
                      sat='CAMS', START_DATE='2020-01-01', END_DATE='2021-01-01')

****** Processing North East in 2020 year *******
prepared Region geographies ...
pulled imagery ...
calculated LSOAs scores ...
running efficient conversion to gdf
running efficient conversion to gdf. data 0 of 1705...
converted feature collection to geopandas df ...
saved csv ...
****** Processing North West in 2020 year *******
prepared Region geographies ...
pulled imagery ...
calculated LSOAs scores ...
running efficient conversion to gdf
running efficient conversion to gdf. data 0 of 4642...
running efficient conversion to gdf. data 2000 of 4642...
running efficient conversion to gdf. data 4000 of 4642...
converted feature collection to geopandas df ...
saved csv ...
****** Processing Yorkshire and The Humber in 2020 year *******
prepared Region geographies ...
pulled imagery ...
calculated LSOAs scores ...
running efficient conversion to gdf
running efficient conversion to gdf. data 0 of 3451...
running efficient conversion to gdf. data 2000 of 3451...
converted feature collecti

In [ ]:
for region_name in regions['RGN22NM'].unique():
    lsoa_OMI_scores = extract_AIQ_lsoa_scores_region(region_name=region_name, ee_regions=ee_regions, ee_lsoas_UK=ee_lsoas_UK,\
                      sat='OMI', START_DATE='2020-01-01', END_DATE='2021-01-01')

****** Processing North East in 2020 year *******
prepared Region geographies ...
pulled imagery ...
calculated LSOAs scores ...
running efficient conversion to gdf
running efficient conversion to gdf. data 0 of 1705...
converted feature collection to geopandas df ...
saved csv ...
****** Processing North West in 2020 year *******
prepared Region geographies ...
pulled imagery ...
calculated LSOAs scores ...
running efficient conversion to gdf
running efficient conversion to gdf. data 0 of 4642...
running efficient conversion to gdf. data 2000 of 4642...
running efficient conversion to gdf. data 4000 of 4642...
converted feature collection to geopandas df ...
saved csv ...
****** Processing Yorkshire and The Humber in 2020 year *******
prepared Region geographies ...
pulled imagery ...
calculated LSOAs scores ...
running efficient conversion to gdf
running efficient conversion to gdf. data 0 of 3451...
running efficient conversion to gdf. data 2000 of 3451...
converted feature collecti

In [ ]:
for region_name in regions['RGN22NM'].unique():
    lsoa_S5_scores = extract_AIQ_lsoa_scores_region(region_name=region_name, ee_regions=ee_regions, ee_lsoas_UK=ee_lsoas_UK,\
                      sat='S5', START_DATE='2020-01-01', END_DATE='2021-01-01')

****** Processing North East in 2020 year *******
prepared Region geographies ...
pulled imagery ...
calculated LSOAs scores ...
running efficient conversion to gdf
running efficient conversion to gdf. data 0 of 1705...
converted feature collection to geopandas df ...
saved csv ...
****** Processing North West in 2020 year *******
prepared Region geographies ...
pulled imagery ...
calculated LSOAs scores ...
running efficient conversion to gdf
running efficient conversion to gdf. data 0 of 4642...
running efficient conversion to gdf. data 2000 of 4642...
running efficient conversion to gdf. data 4000 of 4642...
converted feature collection to geopandas df ...
saved csv ...
****** Processing Yorkshire and The Humber in 2020 year *******
prepared Region geographies ...
pulled imagery ...
calculated LSOAs scores ...
running efficient conversion to gdf
running efficient conversion to gdf. data 0 of 3451...
running efficient conversion to gdf. data 2000 of 3451...
converted feature collecti